In [1]:
import os
os.chdir('../')

In [2]:
import json

class MockAIGame:
    observations = []
    rewards = []
    env = None

    def __init__(self, env):
        self.env = env
    
config = None
with open('config.json') as f:
    config = json.load(f)


In [3]:
import gym

all_observations = []

In [4]:

env = gym.make('CartPole-v1')

steps = []

for i_episode in range(200):
    observation = env.reset()
    for t in range(1000):
        env.render()
#         print(observation)
        all_observations.append(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            steps.append(t+1)
#             print("Episode finished after {} timesteps".format(t+1))
            break

env.close()


In [6]:
len(all_observations)

4212

In [7]:
import pandas as pd

cols = ['pos', 'velocity', 'angle', 'angular velocity']
df = pd.DataFrame(data=all_observations, columns=cols)

In [12]:
df.describe()

,pos,velocity,angle,angular velocity
count,4212.000000,4212.000000,4212.000000,4212.000000
mean,-0.006458,-0.036341,0.005293,0.047520
std,0.068947,0.539895,0.091299,0.820706
min,-0.326433,-2.292229,-0.208935,-2.870586
25%,-0.043662,-0.386120,-0.046157,-0.443360
50%,-0.005977,-0.024303,0.005752,0.025472
75%,0.033438,0.241026,0.057785,0.575758
max,0.494948,2.290255,0.209399,3.011374


In [15]:
means = df.describe().to_numpy()[1]
vstds = df.describe().to_numpy()[2]

In [36]:
means

array([-0.00645789, -0.03634136,  0.00529333,  0.04752025])

In [29]:
vstds

array([0.06894682, 0.53989509, 0.09129902, 0.82070579])

In [61]:
def normalize_f(mean, std):
    def _f(x):
        return (x - mean) / std
    return _f

fs = [normalize_f(means[idx], vstds[idx]) for idx in range(4)]

    
print(fs)

[<function normalize_f.<locals>._f at 0x12debbc20>, <function normalize_f.<locals>._f at 0x12debbdd0>, <function normalize_f.<locals>._f at 0x12debbe60>, <function normalize_f.<locals>._f at 0x12c111f80>]


In [63]:
normalized_observations = []

for obs in all_observations:
    normalized_observations.append([f(val) for f, val in zip(fs, obs)])

print(len(normalized_observations))

4212


In [64]:
dfn = pd.DataFrame(data=normalized_observations, columns=cols)

In [65]:
dfn.describe()

,pos,velocity,angle,angular velocity
count,4.212000e+03,4.212000e+03,4.212000e+03,4.212000e+03
mean,2.699118e-17,-3.373897e-17,1.012169e-17,-1.518254e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-4.640901e+00,-4.178381e+00,-2.346443e+00,-3.555606e+00
25%,-5.395995e-01,-6.478641e-01,-5.635317e-01,-5.981193e-01
50%,6.977980e-03,2.229680e-02,5.028364e-03,-2.686450e-02
75%,5.786425e-01,5.137425e-01,5.749466e-01,6.436389e-01
max,7.272362e+00,4.309349e+00,2.235578e+00,3.611347e+00


In [67]:
from scipy.stats import norm
import numpy as np

In [110]:
# norm.ppf()
EPS = 0
bins = 5
intervals = norm.ppf(np.linspace(EPS, 1.0 - EPS, bins + 1))
print([round(k, 4) for k  in list(intervals)])

[-inf, -0.8416, -0.2533, 0.2533, 0.8416, inf]


In [114]:
intervals[0] < -200002.0

True

In [91]:
np.searchsorted(intervals, -2)

1

In [94]:
len(intervals) == bins + 1

True

In [96]:
np.linspace(-1, 1, 6)

array([-1. , -0.6, -0.2,  0.2,  0.6,  1. ])